In [ ]:
##Arcgis License
import arcpy
from arcpy.sa import *

In [5]:
## set workspace (need to change)
arcpy.env.workspace = 'data/'
arcpy.env.overwriteOutput = True
##load extensions
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("Spatial")

u'CheckedOut'

In [ ]:
def arc_beg(work_path):
    """This function is used to set workspace and load extensions of Arcgis. The input should be the path all files store."""
    arcpy.env.workspace = work_path  # set workspace
    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("3D")  # load 3D Analyst tool
    arcpy.CheckOutExtension("Spatial")  # load Spatial Analyst tool
    return

In [ ]:
#Create LAS Dataset (3D Analyst or Spatial Analyst)
# input can be a folder if choose recursion
arcpy.CreateLasDataset_management('data/Government Hill/1662_2638.las', 'GH.lasd', 'RECURSION')

In [ ]:
#LAS Dataset to Raster (Spatial Analyst or 3D Analyst)
arcpy.LasDatasetToRaster_conversion('GH.lasd', 'GH', "ELEVATION", "BINNING AVERAGE NATURAL_NEIGHBOR",
                                   "FLOAT", "CELLSIZE", 1, 1)

In [ ]:
def las_to_raster(las, dem):
    """This function converts las data to las dataset, and turn the dataset into raster. The input should be .las files or folder and the name of output raster file"""
    lasd = 'dataset.lasd'
    arcpy.CreateLasDataset_management(las, lasd, 'RECURSION')  # convert las data into las dataset
    arcpy.LasDatasetToRaster_conversion(lasd, dem, "ELEVATION", "BINNING AVERAGE NATURAL_NEIGHBOR",
                                        "FLOAT", "CELLSIZE", 1, 1)  # convert las dataset into raster
    return

In [ ]:
#Project dem into different coordinate system
out_coordinate_system = arcpy.SpatialReference('WGS_1984_2')
ProjectRaster_management('GH', )

In [ ]:
#Aspect (3D Analyst or Spatial Analyst)
arcpy.Aspect_3d('GH', 'aspect')
#filter south facing or horizontal aspect. Flat, 112.5 <= aspect <= 247.5, set value to 1, others to None
#outSetNull = SetNull('aspect', 1, "VALUE < 0")
#outSetNull.save("filtered_aspect")
filter_aspect = Con((Raster('aspect') == -1) | (Raster('aspect') >= 112.5) & (Raster('aspect') <= 247.5), 1, '')
# save aspect_raster
filter_aspect.save("filtered_aspect.tif")

In [ ]:
#Slope (3D Analyst or Spatial Analyst)
arcpy.Slope_3d('GH', 'slope', "DEGREE", 1)
# filter slope degree <= 35 to 1, others to None
filter_slope = Con(Raster('slope') <= 35, 1, '')
#save slope_raster
filter_slope.save("filtered_slope.tif")

In [ ]:
#Combine slope and aspect
result = Times("filtered_aspect.tif", "filtered_slope.tif")
result.save("aspect_slope")

In [ ]:
#Combine with raster
result = Times("aspect_slope", "GH")
result.save("result")

In [ ]:
#Extract by building polygon
building_raster = ExtractByMask("result", "data/buildings/buildings.shp")
building_raster.save("braster")

In [3]:
#Area Solar Radiation (Spatial Analyst)
# unit: watt hours per square meter (WH/m2) ??
solar = AreaSolarRadiation('braster', '', '', TimeWholeYear(2018))
solar.save('solar')

In [ ]:
#Extract building polygon
arcpy.RasterToPolygon_conversion('GH', 'building', "NO_SIMPLIFY", "VALUE")

In [ ]:
#Hillshade (3D Analyst or Spatial Analyst)
#Haven't used yet
arcpy.HillShade_3d(raster, hillshaded_raster, 315, 45, "SHADOWS", 1)